In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import sqlite3

%matplotlib inline

In [5]:
db = sqlite3.connect("../data/HMDA.db")
for chunk in pd.read_csv("../data/hmda_2014_nationwide_all-records_labels.csv", 
                         chunksize = 10000, 
                         low_memory = False):
    chunk.to_sql("hmda_2014", db, if_exists="append")


In [6]:
db.close()

In [28]:
def sample(table, n = 20000): 
    """
    Randomly samples a particular table in the sqlite3 database for 
    n observations for various purposes. Runtime is not the best, so only 
    useful for calls for EDA purposes. Using apache spark is best if model 
    building is needed. 
    
    Keyword Arguments: 
    table - the table to pull from typically 'hmda_201X' 
    n - the the number of observations to pull from the dataset 
    """
    con = sqlite3.connect("../data/HMDA.db")
    query = f"""SELECT * FROM {table} ORDER BY RANDOM() LIMIT {n}"""
    df = pd.read_sql(sql = query, con = con)
    con.close()
    
    return df

In [30]:
data_2014 = sample("hmda_2014")


In [44]:
data_2014.columns

Index(['index', 'as_of_year', 'respondent_id', 'agency_name', 'agency_abbr',
       'agency_code', 'loan_type_name', 'loan_type', 'property_type_name',
       'property_type', 'loan_purpose_name', 'loan_purpose',
       'owner_occupancy_name', 'owner_occupancy', 'loan_amount_000s',
       'preapproval_name', 'preapproval', 'action_taken_name', 'action_taken',
       'msamd_name', 'msamd', 'state_name', 'state_abbr', 'state_code',
       'county_name', 'county_code', 'census_tract_number',
       'applicant_ethnicity_name', 'applicant_ethnicity',
       'co_applicant_ethnicity_name', 'co_applicant_ethnicity',
       'applicant_race_name_1', 'applicant_race_1', 'applicant_race_name_2',
       'applicant_race_2', 'applicant_race_name_3', 'applicant_race_3',
       'applicant_race_name_4', 'applicant_race_4', 'applicant_race_name_5',
       'applicant_race_5', 'co_applicant_race_name_1', 'co_applicant_race_1',
       'co_applicant_race_name_2', 'co_applicant_race_2',
       'co_applicant_r

In [45]:
for col in data_2014.columns: 
    print(col, data_2014[col].unique())

index [ 1467299  1237951  4399910 ... 11342625  7872010  7553743]
as_of_year [2014]
respondent_id ['0000019608' '32-0016270' '0000480228' ... '0000022394' '0000010171'
 '0000058262']
agency_name ['Federal Deposit Insurance Corporation'
 'Department of Housing and Urban Development'
 'Consumer Financial Protection Bureau'
 'National Credit Union Administration'
 'Office of the Comptroller of the Currency' 'Federal Reserve System']
agency_abbr ['FDIC' 'HUD' 'CFPB' 'NCUA' 'OCC' 'FRS']
agency_code [3 7 9 5 1 2]
loan_type_name ['Conventional' 'FHA-insured' 'FSA/RHS-guaranteed' 'VA-guaranteed']
loan_type [1 2 4 3]
property_type_name ['One-to-four family dwelling (other than manufactured housing)'
 'Manufactured housing' 'Multifamily dwelling']
property_type [1 2 3]
loan_purpose_name ['Home purchase' 'Refinancing' 'Home improvement']
loan_purpose [1 3 2]
owner_occupancy_name ['Owner-occupied as a principal dwelling'
 'Not owner-occupied as a principal dwelling' 'Not applicable']
owner_occupan